# Poll Data - Web Scraping of Real Clear Politics Site

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#using Pandas to get all tables from website page with Poll Data
dfs = pd.read_html('https://www.realclearpolitics.com/epolls/2020/president/us/general_election_trump_vs_biden-6247.html')

dfs

[                             Poll         Date   Sample  MoE  Biden (D)  \
 0                     RCP Average   7/9 - 7/24       --   --       50.0   
 1         CBS News/YouGovCBS News  7/21 - 7/24  1401 LV   --       51.0   
 2      Rasmussen ReportsRasmussen  7/15 - 7/21  2500 LV  2.0       47.0   
 3        The Hill/HarrisXThe Hill  7/17 - 7/20  2829 RV  1.8       45.0   
 4                FOX NewsFOX News  7/12 - 7/15  1104 RV  3.0       49.0   
 5        ABC News/Wash PostABC/WP  7/12 - 7/15   673 LV   --       54.0   
 6  CNBC/Change Research (D)*CNBC*  7/10 - 7/12  1258 LV  2.8       51.0   
 7            QuinnipiacQuinnipiac   7/9 - 7/13  1273 RV  2.8       52.0   
 8   NBC News/Wall St. JrnlNBC/WSJ   7/9 - 7/12   900 RV  3.3       51.0   
 
    Trump (R)      Spread  
 0       40.9  Biden +9.1  
 1       41.0   Biden +10  
 2       45.0    Biden +2  
 3       38.0    Biden +7  
 4       41.0    Biden +8  
 5       44.0   Biden +10  
 6       41.0   Biden +10  
 7       37.0 

In [3]:
# making dfs for each table on the webpage
import urllib
from bs4 import BeautifulSoup

html_table = urllib.request.urlopen('https://www.realclearpolitics.com/epolls/2020/president/us/general_election_trump_vs_biden-6247.html').read()

# fix HTML
soup = BeautifulSoup(html_table, "html.parser")
# warn! id ratings-table is your page specific
for table in soup.findChildren(attrs={'id': 'ratings-table'}): 
    for c in table.children:
        if c.name in ['tbody', 'thead']:
            c.unwrap()

df = pd.read_html(str(soup), flavor="bs4")
len(df[2])

166

In [4]:
#saving df with poll_data needed
poll_data = df[2]

#saving df to csv file
poll_data.to_csv('poll_data.csv')

In [51]:
poll_data_dates = pd.read_csv('data/poll_data_dates.csv')
poll_data_dates = poll_data_dates.dropna()

poll_data_dates.sort_values(by=['Date'],ascending=True)
poll_data.head()

,Poll,Date,Sample,MoE,Biden (D),Trump (R),Spread
0,RCP Average,7/9 - 7/24,--,--,50.0,40.9,Biden +9.1
1,CBS News/YouGovCBS News,7/21 - 7/24,1401 LV,--,51.0,41.0,Biden +10
2,Economist/YouGovYouGov,7/19 - 7/21,1222 RV,3.2,48.0,41.0,Biden +7
3,Rasmussen ReportsRasmussen,7/15 - 7/21,2500 LV,2.0,47.0,45.0,Biden +2
4,The Hill/HarrisXThe Hill,7/17 - 7/20,2829 RV,1.8,45.0,38.0,Biden +7


In [50]:
#Getting Twitter notebook to add poll data

Tweet_notebook = pd.read_csv('data/Tweet_notebook.csv')
#Tweet_notebook.drop(['Unnamed: 0'],inplace=True)

Tweet_notebook.head()

,Date,username,likes_count,retweet_date,Cases_x,Deaths_x,14 days,Cases_y,Deaths_y,28 days,Cases,Deaths,Sentiment,Total Case Growth,Total Death Growth,Lethality_48,Lethality_14,Lethality_dayof
0,2/11/2020,nytimes,4374,NaN,12,0,2/25/20 0:00,15,0,3/10/20 0:00,711,28,0.136,58.250,inf,0.039,0.000,0.0
1,2/13/2020,tamiann02,0,NaN,13,0,2/27/20 0:00,16,0,3/12/20 0:00,1561,43,0.256,119.077,inf,0.028,0.000,0.0
2,2/13/2020,1val1richy,0,NaN,13,0,2/27/20 0:00,16,0,3/12/20 0:00,1561,43,0.256,119.077,inf,0.028,0.000,0.0
3,2/13/2020,kolsaw,0,NaN,13,0,2/27/20 0:00,16,0,3/12/20 0:00,1561,43,0.061,119.077,inf,0.028,0.000,0.0
4,2/15/2020,_watch_observe_,0,NaN,13,0,2/29/20 0:00,24,1,3/14/20 0:00,2870,58,0.217,219.769,58.0,0.020,0.042,0.0


In [47]:
# merging to get poll data
# In theory, the merge_asof should allow for the poll data to be matched via date to the tweet notebook
# by the closest date. However, I keep getting an error message saying that keys must be sorted. I checked for
# nulls and even sort the keys in the function.

right = poll_data_dates
left = Tweet_notebook

pd.merge_asof(left,right.sort_values(by='Date'),on='Date',allow_exact_matches=False,direction='nearest')



ValueError: left keys must be sorted